In [22]:
import tensorflow as tf
import os
from transformers import BertTokenizer, TFBertForQuestionAnswering 
from transformers import BertModel
import warnings
warnings.filterwarnings("ignore")

In [23]:
modelName = 'bert-large-uncased-whole-word-masking-finetuned-squad'
tokenizer = BertTokenizer.from_pretrained(modelName)
model = TFBertForQuestionAnswering.from_pretrained(modelName)
print(model)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [25]:
def process_data(text, question):
    input_text =  question + " [SEP] " + text 
    input_ids = tokenizer.encode(input_text)
    input = tf.constant(input_ids)[None, :]  # Batch size 1
    input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
    answer=model(input, token_type_ids = tf.convert_to_tensor([token_type_ids]))
    answer_start_index1 = int(tf.math.argmax(answer.start_logits, axis=-1)[0])
    answer_end_index1 = int(tf.math.argmax(answer.end_logits, axis=-1)[0]) 
    end_answer=(" ".join(input_tokens[answer_start_index1 :answer_end_index1 + 1]))
    end_answer = "".join(end_answer.split()).replace('#','')
    return end_answer, input_tokens,token_type_ids

In [35]:
db_content="output.txt"
file = open(db_content, encoding="utf8")
data_value=file.read()
text=data_value[:2000]
question =r"""What is the ISIN of Deutsche Börse AG"""
print("-"*50)
print(question)
end_answer, input_tokens,token_type_ids=process_data(text,question)
print(end_answer)
#print(input_tokens)
#print(token_type_ids)
question =r"""Who is the Chairman of the Board of Deutsche Börse AG"""
print("-"*50)
print(question)
end_answer, input_tokens,token_type_ids=process_data(text,question)
print(end_answer)
#print(input_tokens)
#print(token_type_ids)

--------------------------------------------------
What is the ISIN of Deutsche Börse AG
de0005810055
--------------------------------------------------
Who is the Chairman of the Board of Deutsche Börse AG
theodorweimer
